In [1]:
import os
import cv2
import glob
import numpy as np
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
basedir = "/input/"
model_image_size = 224


print("-------- loading train data")
X_train = list()
y_train = list()
for i in range(10):
    dir = os.path.join(basedir, "train", "c%d"%i)
    image_files = glob.glob(os.path.join(dir,"*.jpg")) #返回F:\pose_recognication\data\train\ci文件夹中所有以jpg结尾的文件的路径
    print("loding {}, image count={}".format(dir, len(image_files)))
   #data_files=glob.glob("data/*.txt")

#         glob.glob()返回的是列表 list类型。是所有路径下的符合条件的文件名的列表。

#        此例中参数为相对路径，指simple_httpd.py  web服务器当前目录下的data文件夹下的所有txt文件。

#        也可以为绝对路径
    for image_file in image_files:
        image = cv2.imread(image_file)
        X_train.append(cv2.resize(image, (model_image_size, model_image_size)))
        label = np.zeros(10, dtype=np.uint8)
        label[i]=1
        y_train.append(label)
X_train = np.array(X_train)
y_train = np.array(y_train)
        
print("-------- loading valid data")
X_valid = list()
y_valid = list()
for i in range(10):
    dir = os.path.join(basedir, "valid", "c%d"%i)
    image_files = glob.glob(os.path.join(dir,"*.jpg"))
    print("loding {}, image count={}".format(dir, len(image_files)))
    for image_file in image_files:
        image = cv2.imread(image_file)
        X_valid.append(cv2.resize(image, (model_image_size, model_image_size)))
        label = np.zeros(10, dtype=np.uint8)
        label[i]=1
        y_valid.append(label)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)

-------- loading train data
loding /input/train/c0, image count=1707
loding /input/train/c1, image count=2113
loding /input/train/c2, image count=868
loding /input/train/c3, image count=1691
loding /input/train/c4, image count=1805
loding /input/train/c5, image count=1976
loding /input/train/c6, image count=1728
loding /input/train/c7, image count=1300
loding /input/train/c8, image count=722
loding /input/train/c9, image count=1596
-------- loading valid data
loding /input/valid/c0, image count=92
loding /input/valid/c1, image count=158
loding /input/valid/c2, image count=247
loding /input/valid/c3, image count=345
loding /input/valid/c4, image count=388
loding /input/valid/c5, image count=364
loding /input/valid/c6, image count=328
loding /input/valid/c7, image count=298
loding /input/valid/c8, image count=108
loding /input/valid/c9, image count=316


In [4]:
#base_model = VGG16(input_tensor=Input((model_image_size, model_image_size, 3)), weights='imagenet', include_top=False)

#base_model = InceptionV3(input_tensor=Input((model_image_size, model_image_size, 3)), weights='imagenet', include_top=False)

fine_tune_layer = 116
final_layer = 135
visual_layer = 132
batch_size = 64
base_model = Xception(input_tensor=Input((model_image_size, model_image_size, 3)), weights='imagenet', include_top=False)
for layers in base_model.layers:
    layers.trainable = False


x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.25)(x)
x = Dense(10, activation='softmax')(x)
model = Model(base_model.input, x)
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
print("total layer count {}".format(len(base_model.layers)))

for i in range(fine_tune_layer):
    model.layers[i].trainable = False

#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("done")

total layer count 132
done


In [5]:
model.fit(X_train, y_train, batch_size=4, epochs=10, validation_data=(X_valid, y_valid))
#model.save("/data/inception-not-finetune.h5")

Train on 15506 samples, validate on 2644 samples
Epoch 1/10
15506/15506 [==============================] - 109s 7ms/step - loss: 0.0985 - acc: 0.9676 - val_loss: 2.8227 - val_acc: 0.8239
Epoch 2/10
15506/15506 [==============================] - 106s 7ms/step - loss: 0.0380 - acc: 0.9898 - val_loss: 2.7109 - val_acc: 0.8239
Epoch 3/10
15506/15506 [==============================] - 106s 7ms/step - loss: 0.0300 - acc: 0.9915 - val_loss: 2.8175 - val_acc: 0.8239
Epoch 4/10
15506/15506 [==============================] - 107s 7ms/step - loss: 0.0243 - acc: 0.9929 - val_loss: 2.8200 - val_acc: 0.8239
Epoch 5/10
15506/15506 [==============================] - 107s 7ms/step - loss: 0.0223 - acc: 0.9935 - val_loss: 2.8229 - val_acc: 0.8239
Epoch 6/10
15506/15506 [==============================] - 106s 7ms/step - loss: 0.0211 - acc: 0.9937 - val_loss: 2.8174 - val_acc: 0.8239
Epoch 7/10
15506/15506 [==============================] - 106s 7ms/step - loss: 0.0191 - acc: 0.9944 - val_loss: 2.8187 - v

In [ ]:
model.save("/data/Xception-not-finetune.h5")